<a href="https://colab.research.google.com/github/leegyuhi/oooooooooooooh/blob/master/xu2017seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Indtroduction

화학 특성을 자동으로 예측하기 위해 기계 학습 방법을 사용하는 것은 최근 약물 발견 커뮤니티에 큰 관심을 불러일으켰다. 그러나 대부분의 기계 학습 알고리즘은 고정된 길이의 연속 특성 벡터를 입력으로 사용한다. 따라서 분자들을 고정된 길이의 벡터로 표현하는 방법이 필요하다. 

전통적으로 분자를 고정된 길이의 벡터로 표현하기 위해 사용되던 방법은 'fingerprint'이다. 이 방법은 속도면에서 효율적이지만, 이미지 영역에서 이해가 떨어지고 다시 분자 구조로 전환할 수 없다는 단점이 있다.

이러한 문제를 해결하기 위해 'seq2seq fingerprint'를 사용한다. seq2seq fingerprint는 비지도 데이터와 딥러닝에 기반하는 분자 지문 방법이다. 이 방법은 전문적인 지식을 요구하지 않는 데이터를 기반으로 하고, 다시 분자 표현 형태로 전환할 수 있으며, 충분한 양의 데이터를 보장한다는 장점이 있다. 또한 레이블이 없기 때문에 고도의 인력과 비용을 필요로 하지 않는다.

seq2seq 학습의 입력과 출력은 각각 일련의 SMILE과 분자의 텍스트 표현이다. 일련의 SMILE과 고정된 길이의 벡터를 연결시키고, 다시 원래의 SMILE로 번역한다. 여기서 벡터를 seq2seq fingerprint로 추출한다. 모델이 잘 훈련된다면, 특성 벡터가 원본 분자의 모든 정보를 코드화 했다고 볼 수 있다. 따라서 seq2seq fingerprint는 엄밀한 정보를 모두 담고 있다고 추측하고, 이를 통해 분자의 특성을 예측할 수 있다. 

> 논문의 요약
> 1. 훈련 과정을 가속화하기 위해 LSTM 대신 GRU 셀을 사용한다.
> 2. 주의 메커니즘은 지문 공간을 중앙 집중화하기 위해 사용된다.
> 3. fingerprint를 꺼내기 위해 fingerprint extraction layer set을 추가한다.

## 2. Methodology

2절에서는 제안된 방법의 전체 파이프라인을 제시한다.
첫째로 fingerprint 방법의 핵심이 소개되어 전체 파이프라인을 개괄한다. 둘째, 우리는 우리의 방법의 각 단계와 원래의 seq2seq 학습 방법에 대한 개선과 적응을 상세히 기술한다. 마지막으로, 이 섹션을 끝내기 위한 방법에 대해 논의한다.

### 2.1 Overview
![xu2017_02](https://user-images.githubusercontent.com/52767505/70285828-3371ed80-180c-11ea-9643-cd01cb90784b.PNG)

첫번째로 seq2seq fingerprint 모델을 레이블이 없는 방대한 양의 샘플로 훈련시킨다. 그 다음 훈련된 모델로 레이블이 있는 데이터에서 seq2seq fingerprint를 생성한다. 예측 모델을 훈련하기 위해, 생성된 fingerprint와 레이블이 있는 데이터를 지도 학습 방법에 적용시킨다.

이러한 방법은 방대한 양의 레이블이 없는 데이터에서 상대적으로 적은 레이블이 있는 데이터에 대한 지도 훈련으로 지식을 전달할 수 있도록 한다. 즉, 최종적으로 예측 모델의 성능을 향상시키도록 한다.

비지도 seq2seq fingerprint 방법의 핵심은 입력과 출력을 SMILE로 설정한다는 것이다. 즉, SMILE 스트링을 SMILE로 번역한다. 이러한 과정의 중간 단계에서 고정된 길이의 벡터를 fingerprint로 추출한다.

### 2.2 Unsupervised Seq2seq Training
![xu2017_03](https://user-images.githubusercontent.com/52767505/70286669-6d90be80-180f-11ea-94fa-246ca4de3316.PNG)

Fingerprint 생성기를 훈련시키기 위해 두 가지 네트워크를 이용한다. 먼저 SMILE 스트링과 고정된 길이의 벡터를 연관시키기 위해 'perceiver network'라는 GRU network를 사용한다. 그 다음 'interpreter network'라는 GRU network를 사용하여 seq2seq fingerprint를 다시 원래의 SMILE 스트링으로 번역한다.

LSTM의 훈련 속도가 느리기 때문에, 보다 속도가 빠르지만 성능이 동일한 GRU를 사용한다. Attention mechanism을 이용하여 perceiver와 interpreter 간의 연결을 강화시켜서 두 네트워크 사이 은닉 상태를 더 잘 공유하도록 한다. 입력과 출력 게이트에 dropout layer를 추가하여 일반화 성능을 향상시키고 과대적합을 방지한다.

### 2.3 Fingerprint Extraction
![xu2017_04](https://user-images.githubusercontent.com/52767505/70289860-81412280-1819-11ea-840c-f91610bcaebf.PNG)

원본 seq2seq 모델은 명확한 임베딩 벡터를 출력하지 않기 때문에, fingerprint 추출에 어려움이 발생한다. GRU 셀을 perceiver network와 interpreter network 사이에 삽입하여 seq2seq fingerprint를 추출하도록 했다.

### 2.4 Discussion
이러한 방법은 실제로 unlabeled data에서 labeled data 훈련으로 지식을 이전할 수 있다. 그러나 unlabeled data는 지도 훈련에서 직접 사용되지 않기 때문에 기술적으로 부분지도 되지는 않는다. 따라서 seq2seq fingerprint 방법을 비지도 되었다고 명명할 수 있다.

## 3. Discussion
- 실험의 설정 : 데이터 세트, 하드웨어와 소프트웨서 설정
- seq2seq fingerprint 방법의 회복 수행 : SMILE 번역
- 두 가지 예측 작업에 대한 우수한 성과

### 3.1 Experiment Setup
LogP와 PM2라는 데이터셋을 이용하여 훈련했다. 훈련 세트는 총 334,092개의 유효분자 SMILE 표현이다. LogP 데이터셋은 SMILE 스트링과 물-옥탄올 분배계수의 쌍으로 이루어져있다. 분배계수 LogP < 1.88인 샘플은 negative로, 나머지는 positive로 분류했다. PM2-10k 데이터셋은 SMILE 스트링과 이진 비규칙 클래스 레이블의 쌍으로 이루어져있다.

Seq2seq fingerprint method는 텐서플로우 패키지에서 시행했다. Intel i7 6700K @ 4.00 GHz CPU, 16 Gigabytes RAM, Nvidia GTX 1080 GPU 환경에서 훈련 모델을 훈련시켰다. 

### 3.2 Seq2seq Fingerprint Recovery Performance
특성 벡터의 길이가 512, 768, 1024인 세가지 seq2seq fingerprint에 대해 24시간 동안 실험했다. 세가지 모델은 GRU 층의 갯수에서만 차이를 보인다. 

![xu2017_05](https://user-images.githubusercontent.com/52767505/70312861-cbe18f80-1857-11ea-9cab-b0dbb4fd7ef7.PNG)

회복 성능은 perplexity와 Exact Match accuracy를 통해 평가된다. 테이블에서 GRU 층이 증가할 수록 회복 성능이 감소하는 것을 볼 수 있다. seq2seq fingerprint의 길이가 길어질수록 더 많은 시간이 필요하기 때문이다. 또한 길이가 길수록 분자의 표현 공간이 확장되어 더 많은 null space를 남기게 된다. 따라서 fingerprint의 길이가 길어질수록 수행능력이 떨어진다고 할 수 없다.

### 3.3 LogP Solubility and PM2 Promiscuous Classification
![xu2017_06](https://user-images.githubusercontent.com/52767505/70313744-84f49980-1859-11ea-943e-e39a5041283d.PNG)

위의 그림은 fingerprint의 길이가 정확도에 미치는 영향을 보여준다. seq2seq fingerprint 모델이 circular와 neural fingerprint의 성능을 훨씬 크게 뛰어넘는 것을 보여준다. 

## 4. Conclusion

이 논문에서는 seq2seq fingerprint로 불리는 새로운 비지도 분자 표현 시스템에 대해서 논의한다. 영어를 프랑스어로 번역하는 모델인 sequence to sequence 학습 모델에 기반한다. 이 모델은 분자의 SMILE 스트링을 SMILE로 변환함과 동시에 고정된 길이의 fingerprint 벡터를 생성한다. 분류 작업에 관한 실험에서 뛰어난 성능을 나타낸다. 또한, 데이터 기반 라벨이 없는 모델의 특성은 우리에게 훨씬 더 많은 이점을 가져다 준다. seq2seq fingerprint 시스템은 완전한 비지도 시스템이기 때문에, 고비용의 레이블 수집 과정이 필요없다. 즉, 각각의 유효한 분자를 이용할 수 있으며, 이론적으로는 무한의 양에 도달한다. 매우 제한된 데이터 샘플로 훈련한 지도학습모델과는 다르게, seq2seq fingerprint는 충분히 많은 양의 샘플로 훈련하기 때문에 보다 특정한 분야에 강하다.